In [34]:
import json
import pandas as pd
import numpy as np
from data_crawler.stock_data import ticker_crawler

In [ ]:
LOCATE = 'stock_market_forecasting_ai/data'
PERIOD = '5y'

ticker_crawler(LOCATE, PERIOD)

In [29]:
df = pd.read_csv('../../data.csv')
df.head(10)

,Price,Close,Dividends,High,Low,Open,Stock Splits,Volume
0,Ticker,PETR4.SA,PETR4.SA,PETR4.SA,PETR4.SA,PETR4.SA,PETR4.SA,PETR4.SA
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-11,6.778051376342773,0.0,6.920248693009845,6.635854624715365,6.917286190054316,0.0,93072900
3,2020-11-12,6.490695953369141,0.0,6.760277531109861,6.449222038344326,6.736578070375723,0.0,77275000
4,2020-11-13,6.703991413116455,0.0,6.7543528401588215,6.48477124982608,6.523283228467752,0.0,66065200
5,2020-11-16,6.899510383605957,0.0,7.000232639132881,6.85211091065742,6.8728484272772095,0.0,101583300
6,2020-11-17,7.018008708953857,0.0,7.151317945609021,6.807675529062019,6.828412483761291,0.0,94082100
7,2020-11-18,6.976535320281982,0.0,7.148356555296221,6.9765353202819815,7.106882639443044,0.0,68540400
8,2020-11-19,7.056520462036133,0.0,7.092069932580247,6.923211218834948,6.9765345770916225,0.0,65382100
9,2020-11-23,7.435713291168213,0.0,7.435713291168213,7.121695283289703,7.124657221583883,0.0,125649000


In [30]:
new_columns = ['Date', 'Close', 'Dividends', 'High', 'Low', 'Open', 'Stock Splits', 'Volume']
df.columns = new_columns
df.head()

,Date,Close,Dividends,High,Low,Open,Stock Splits,Volume
0,Ticker,PETR4.SA,PETR4.SA,PETR4.SA,PETR4.SA,PETR4.SA,PETR4.SA,PETR4.SA
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-11-11,6.778051376342773,0.0,6.920248693009845,6.635854624715365,6.917286190054316,0.0,93072900
3,2020-11-12,6.490695953369141,0.0,6.760277531109861,6.449222038344326,6.736578070375723,0.0,77275000
4,2020-11-13,6.703991413116455,0.0,6.7543528401588215,6.48477124982608,6.523283228467752,0.0,66065200


In [31]:
df['Ticker'] = df.iloc[0][1]

/tmp/ipykernel_255440/391629775.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['Ticker'] = df.iloc[0][1]


In [32]:
df = df.drop(df.index[[0, 1]])
df.head()

,Date,Close,Dividends,High,Low,Open,Stock Splits,Volume,Ticker
2,2020-11-11,6.778051376342773,0.0,6.920248693009845,6.635854624715365,6.917286190054316,0.0,93072900,PETR4.SA
3,2020-11-12,6.490695953369141,0.0,6.760277531109861,6.449222038344326,6.736578070375723,0.0,77275000,PETR4.SA
4,2020-11-13,6.703991413116455,0.0,6.7543528401588215,6.48477124982608,6.523283228467752,0.0,66065200,PETR4.SA
5,2020-11-16,6.899510383605957,0.0,7.000232639132881,6.85211091065742,6.8728484272772095,0.0,101583300,PETR4.SA
6,2020-11-17,7.018008708953857,0.0,7.151317945609021,6.807675529062019,6.828412483761291,0.0,94082100,PETR4.SA
